In [ ]:
!pip install googlesearch-python

from googlesearch import search
import requests
from bs4 import BeautifulSoup
import re
import json
import time
from IPython.display import FileLink

def limpiar_nombre(raw_title):
    if not raw_title:
        return "Nombre no encontrado"
    nombre = raw_title.strip()

    # Eliminar textos comunes no deseados y truncar tras ciertos separadores
    nombre = re.split(r'[-–—|:•]', nombre)[0]
    nombre = re.sub(r'\b(contacto|contactos|página oficial|home|portal|web|site|oficial)\b', '', nombre, flags=re.I)
    nombre = re.sub(r'\d{1,4}[\s,.-]*[a-zA-Z]*', '', nombre)
    nombre = nombre.strip()

    # Capitalizar cada palabra
    nombre = ' '.join(w.capitalize() for w in nombre.split())

    if not nombre:
        return "Nombre no encontrado"
    return nombre

busquedas = [
  "contacto correo colegio fiscal ecuador",
    "escuela pública contacto email ecuador",
    "colegio municipal ecuador email de contacto",
    "unidad educativa católica ecuador correo",
    "colegio bilingüe contacto correo ecuador",
    "colegio laico ecuador email oficial",
    "escuela técnica ecuador correo contacto",
    "colegio nacional ecuador correo electrónico",
    "colegios rurales contacto email ecuador",
    "colegio privado cristiano ecuador correo",
    "colegios de guayaquil email de contacto",
    "colegios de quito email contacto oficial",
    "instituciones educativas ecuador correo oficial",
    "colegios con página web ecuador email",
    "unidad educativa particular con correo ecuador",
    "colegio con email institucional ecuador",
    "escuelas bilingües ecuador contacto correo",
    "colegios de bachillerato ecuador email",
    "colegios técnicos ecuador contacto correo",
    "institución educativa rural ecuador email contacto",
    "colegio parroquial ecuador correo electrónico",
    "colegio privado bilingüe ecuador email contacto",
    "colegios con correo oficial quito y guayaquil",
    "escuelas fiscales ecuador contacto email",
    "colegios con correo institucional en ecuador",
]

resultados = []
correos_vistos = set()
MAX_RESULTADOS = 1000

for query in busquedas:
    print(f"Buscando: {query}")
    try:
        for url in search(query, num_results=20, lang="es"):
            if len(resultados) >= MAX_RESULTADOS:
                break
            try:
                res = requests.get(url, timeout=10)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")
                texto = soup.get_text(separator=' ')
                # Extraer solo correos terminados en .edu.ec
                correos_raw = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.(edu\.ec)", texto, flags=re.I)
                correos = set(m.group(0) if hasattr(m,'group') else m for m in re.finditer(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.(edu\.ec)", texto, flags=re.I))
                correos = {c.group(0).lower() for c in re.finditer(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.edu\.ec", texto, flags=re.I)}
                correos = {c for c in correos if c not in correos_vistos}

                if not correos:
                    continue

                nombre_raw = soup.title.string if soup.title else None
                nombre = limpiar_nombre(nombre_raw)

                # Evitar que agregue si ya existe nombre igual (puedes optar si quieres unir correos)
                if any(r['nombre'] == nombre for r in resultados):
                    # Si ya está, agregar correos únicos a ese registro
                    for r in resultados:
                        if r['nombre'] == nombre:
                            nuevos = [c for c in correos if c not in r['correos'].split(', ')]
                            if nuevos:
                                r['correos'] += ', ' + ', '.join(nuevos)
                                correos_vistos.update(nuevos)
                    continue

                correos_vistos.update(correos)

                resultado = {
                    "nombre": nombre,
                    "correos": ", ".join(sorted(correos))
                }
                resultados.append(resultado)
                print(f"  + {nombre}: {len(correos)} correos")
                time.sleep(1)
            except Exception as e:
                print(f"Error en {url}: {e}")
    except Exception as e:
        print(f"Error en búsqueda '{query}': {e}")
    if len(resultados) >= MAX_RESULTADOS:
        break

print(f"\nTotal resultados: {len(resultados)}")

# Guardar en JSON
nombre_archivo = "colegios_ecuador.json"
with open(nombre_archivo, "w", encoding="utf-8") as f:
    f.write("[\n")
    for i, item in enumerate(resultados):
        coma = "," if i < len(resultados)-1 else ""
        json_str = json.dumps(item, ensure_ascii=False)
        f.write(f"  {json_str}{coma}\n")
    f.write("]\n")

print(f"\nArchivo '{nombre_archivo}' guardado con éxito.")

# Mostrar enlace para descargar en Colab
display(FileLink(nombre_archivo))

# Mostrar una muestra de resultados en consola (los 10 primeros)
print("\nEjemplo de resultados (10 primeros):")
for item in resultados[:10]:
    print(json.dumps(item, ensure_ascii=False, indent=2))


Buscando: contacto correo colegio fiscal ecuador
Error en https://www.gob.ec/gaddmq/tramites/inscripcion-institucion-educativa-municipal-ordinaria: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error en https://educarecuador.gob.ec/: HTTPSConnectionPool(host='educarecuador.gob.ec', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f0fee50f790>, 'Connection to educarecuador.gob.ec timed out. (connect timeout=10)'))
Error en https://www.gob.ec/mineduc/tramites/pase-transferencia-estudiantes-instituciones-educativas-fiscales: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error en https://www.educarecuador.gob.ec/anexos/correo/manual-uso-correo.pdf: HTTPSConnectionPool(host='www.educarecuador.gob.ec', port=443): Max retries exceeded with url: /anexos/correo/manual-uso-correo.pdf (Caused by ConnectTimeoutError(<urllib3.connec

/content/colegios_ecuador.json


Ejemplo de resultados (10 primeros):
{
  "nombre": "Colegio San Gabriel",
  "correos": "contacto@csgabriel.edu.ec"
}
{
  "nombre": "► Correo Institucional Mail.educacion.gob.ec Outlook",
  "correos": "marta.bonilla@estudiantes.edu.ec, nombre.apellido@estudiantes.edu.ec"
}
{
  "nombre": "Y Ubicación",
  "correos": "adquisiciones@caq.edu.ec, almacen@caq.edu.ec, analista.compensaciones@caq.edu.ec, analista.seleccion@caq.edu.ec, apple.helpdesk@caq.edu.ec, asistente.gerencia@caq.edu.ec, asistente.recgeneral@caq.edu.ec, asistente.secundaria@caq.edu.ec, biblioteca@caq.edu.ec, cobranzas@caq.edu.ec, content.manager@caq.edu.ec, coordinador.transporte@caq.edu.ec, dpto.medico@caq.edu.ec, enfermeria.kinder@caq.edu.ec, extracurricular@caq.edu.ec, help.desk@caq.edu.ec, info@caq.edu.ec, jefe.comunicacion@caq.edu.ec, jefe.personal@caq.edu.ec, jefe.seguridad@caq.edu.ec, musica_cultura@caq.edu.ec, recepcion@caq.edu.ec, secretaria.kinder@caq.edu.ec, secretaria.primaria@caq.edu.ec, secretaria.secundaria@c